<a href="https://colab.research.google.com/github/CZ-CE3004-MDP-Group10/rpi-cv/blob/main/TensorFlow_RPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Directory and Pathing Structure

## Mount Repository
Start by mounting the repository, this repository contains the basic file structure

In [1]:
! git clone https://github.com/CZ-CE3004-MDP-Group10/rpi-cv

fatal: destination path 'rpi-cv' already exists and is not an empty directory.


## Configure Environment Paths

In [2]:
# copy labeled images and its XML files in PASCAL VOC 
%cp -a /content/rpi-cv/TensorFlow /content

In [3]:
TENSORFLOW_PATH = '/content/TensorFlow'
TF_API_MODEL_PATH = TENSORFLOW_PATH + '/models' 
SCRIPTS_PATH = TENSORFLOW_PATH + '/scripts'
WORKSPACE_PATH = TENSORFLOW_PATH + '/workspace'
ANNOTATIONS_PATH = WORKSPACE_PATH + '/annotations'
EXPORTED_MODELS_PATH = WORKSPACE_PATH + '/exported-models'
MODEL_PATH = WORKSPACE_PATH + '/models'
IMAGES_PATH = WORKSPACE_PATH +'/images'
PRE_TRAINED_MODELS_PATH = WORKSPACE_PATH +'/pre-trained-models'

## Mount TensorFlow Model Garden
In order to use the TensorFlow Object Detection API, we need to clone it's GitHub Repo.

In [4]:
import os
os.chdir(TENSORFLOW_PATH)
os.getcwd()

FileNotFoundError: ignored

In [ ]:
!git clone https://github.com/tensorflow/models.git

# Install Required Libraries and Tools
With colab, TensorFlow is already pre-installed along with its other dependencies. 

However, the TensorFlow Object Detection API relies on what are called protocol buffers (also known as protobufs). Protobufs are a language neutral way to describe information. That means you can write a protobuf once and then compile it to be used with other languages, like Python, Java or C.

In [ ]:
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

## Compile and Install the Protobuf libraries
The protoc command used below is compiling all the protocol buffers in the object_detection/protos folder for Python. Environment

To use the object detection api we need to add it to our PYTHONPATH along with slim which contains code for training and evaluating several widely used Convolutional Neural Network (CNN) image classification models.

In [ ]:
import os
os.chdir(TENSORFLOW_PATH + '/models/research/')
os.getcwd()

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

## Set the Tensrflow Object Detection Environment

In [ ]:
import os
import sys
os.environ['PYTHONPATH'] += ":/content/TensorFlow-RPI/models"
print(os.environ['PYTHONPATH'])

sys.path.append('/content/TensorFlow-RPI/models/research')
print(sys.path)

## Build and Install the TensorFlow Object Detection API

In [ ]:
!python slim/setup.py build
!python slim/setup.py install

## Test Tensorflow 2 Object Detection API

In [ ]:
#cd into 'TensorFlow/models/research/object_detection/builders/'
%cd '/content/TensorFlow-RPI/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

# Model Training

## 1. Label Image with Imagelabel
Split 80/20 image for training and testing. Then label the image according to their appropriate signs.

We used https://github.com/tzutalin/labelImg the image labeling

## 2. Create Label Map
TensorFlow requires a label map, which namely maps each of the used labels to an integer values. This label map is used both by the training and detection processes.

In [ ]:
labels = [
    {'name':'0', 'id':1}, 
    {'name':'6', 'id':2}, 
    {'name':'7', 'id':3}, 
    {'name':'8', 'id':4}, 
    {'name':'9', 'id':5}, 
    {'name':'circle', 'id':6},
    {'name':'up', 'id':7}, 
    {'name':'down', 'id':8}, 
    {'name':'left', 'id':9}, 
    {'name':'right', 'id':10}, 
    {'name':'v', 'id':11}, 
    {'name':'w', 'id':12}, 
    {'name':'x', 'id':13}, 
    {'name':'y', 'id':14}, 
    {'name':'z', 'id':15}
]

with open(ANNOTATIONS_PATH+'/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# !cat label_map.pbtxt

## 3. Create TensorFlow Records

Now that we have generated our annotations and split our dataset into the desired training and testing subsets, it is time to convert our annotations into the so called TFRecord format.
Convert *.xml to *.record

To do this we can write a simple script that iterates through all *.xml files in the training_demo/images/train and training_demo/images/test folders, and generates a *.record file for each of the two. Here is an example script that allows us to do just that:

In [ ]:
# Create train data:
!python {SCRIPTS_PATH + '/preprocessing/generate_tfrecord.py'} -x {IMAGES_PATH + '/train'} -l {ANNOTATIONS_PATH + '/label_map.pbtxt'} -o {ANNOTATIONS_PATH + '/train.record'}
# Create test data:
!python {SCRIPTS_PATH + '/preprocessing/generate_tfrecord.py'} -x {IMAGES_PATH + '/test'} -l {ANNOTATIONS_PATH + '/label_map.pbtxt'} -o {ANNOTATIONS_PATH + '/test.record'}

## 4. Download Pre-Trained Model
To begin with, we need to download the latest pre-trained network for the model we wish to use. 

This can be done by simply clicking on the name of the desired model in the table found in [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Clicking on the name of your model should initiate a download for a `*.tar.gz` file.

Once the `*.tar.gz` file has been downloaded, open it using a decompression program of your choice (e.g. 7zip, WinZIP, etc.). Next, open the `*.tar` folder that you see when the compressed folder is opened, and extract its contents inside the folder `/workspace/pre-trained-models`. 

In [ ]:
import os
# import urllib.request
import requests
import tarfile

from shutil import copyfile


os.chdir(PRE_TRAINED_MODELS_PATH)
base_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/"
models = [
          "ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz",
          # "faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz"
]

for model in models:
  url = base_url + model
  r = requests.get(url, allow_redirects=True)
  open(model, 'wb').write(r.content)
  tar = tarfile.open(model)
  tar.extractall()
  tar.close()

## 4. Copy Choosen Model to Training folder
Finally, the object detection training pipeline must be configured. It defines which model and what parameters will be used for training. This is the last step before running training!

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet_v2'

In [ ]:
import os
from shutil import copyfile

os.chdir(WORKSPACE_PATH)
os.mkdir(MODEL_PATH + '/' + CUSTOM_MODEL_NAME)
copyfile(PRE_TRAINED_MODELS_PATH+'/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config', MODEL_PATH + '/' + CUSTOM_MODEL_NAME + '/pipeline.config')

In [ ]:
!cat /content/TensorFlow-RPI/workspace/models/my_ssd_mobnet_v2/pipeline.config

## 5. Updating Choosen Model Config file
There will be multiple parameters that needs to be configured for the config file prior to training. We will use a Python script to shorten the process.



In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [ ]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
# Number of label classes
pipeline_config.model.ssd.num_classes = 15
# Number of images
pipeline_config.train_config.batch_size = 15 
# Model checkpoint (Checkpoints capture the exact value of all parameters (tf.Variable objects) used by a model)
pipeline_config.train_config.fine_tune_checkpoint = PRE_TRAINED_MODELS_PATH + '/ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/ckpt-0' 
# Specify to train DETECTION model
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
# Label map for train
pipeline_config.train_input_reader.label_map_path= ANNOTATIONS_PATH + '/label_map.pbtxt'
# TF records of train
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATIONS_PATH + '/train.record']
# Label map for test
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATIONS_PATH + '/label_map.pbtxt'
# TF reocrds of test
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATIONS_PATH + '/test.record']

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [ ]:
!cat /content/TensorFlow-RPI/workspace/models/my_ssd_mobnet_v2/pipeline.config

## Train the Model
Note TensorFlow 2 is used
Number of training steps can be configured to suit the accuracy

In [ ]:
# --num_train_steps=5000
print("""!python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config """.format(TF_API_MODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

In [ ]:
# run the printed statement ^^^^
!python /content/TensorFlow-RPI/models/research/object_detection/model_main_tf2.py --model_dir=/content/TensorFlow-RPI/workspace/models/my_ssd_mobnet_v2 --pipeline_config_path=/content/TensorFlow-RPI/workspace/models/my_ssd_mobnet_v2/pipeline.config 
